In [1]:
#Implementation of Shamir Secret Sharing Scheme 

import random 
from math import ceil 
from decimal import *


global field_size 
field_size = 10**5

def reconstructSecret(shares): 

	# Combines shares using 
	# Lagranges interpolation. 
	# Shares is an array of shares 
	# being combined 
	sums, prod_arr = 0, [] 
	
	for j in range(len(shares)): 
		xj, yj = shares[j][0],shares[j][1] 
		prod = Decimal(1) 

		for i in range(len(shares)): 
			xi = shares[i][0] 
			if i != j: prod *= Decimal(Decimal(xi)/(xi-xj)) 
				
		prod *= yj 
		sums += Decimal(prod) 
		
	return int(round(Decimal(sums),0)) 

def polynom(x,coeff): 

	# Evaluates a polynomial in x 
	# with coeff being the coefficient 
	# list 
	return sum([x**(len(coeff)-i-1) * coeff[i] for i in range(len(coeff))]) 

def coeff(t,secret): 

	# Randomly generate a coefficient 
	# array for a polynomial with 
	# degree t-1 whose constant = secret''' 
	coeff = [random.randrange(0, field_size) for _ in range(t-1)] 
	coeff.append(secret) 
	
	return coeff 

def generateShares(n,m,secret): 

	# Split secret using SSS into 
	# n shares with threshold m 
	cfs = coeff(m,secret) 
	shares = [] 

	for i in range(1,n+1): 
		r = random.randrange(1, field_size) 
		shares.append([r, polynom(r,cfs)]) 
	
	return shares 


# Driver code 
if __name__ == '__main__': 

	# (3,5) sharing scheme 
	t,n = 3, 5
	secret = 1234
	print('Original Secret:', secret) 

	# Phase I: Generation of shares 
	shares = generateShares(n, t, secret) 
	print('\nShares:', *shares) 

	# Phase II: Secret Reconstruction 
	# Picking t shares randomly for 
	# reconstruction 
	pool = random.sample(shares, t) 
	print('\nCombining shares:', *pool) 
	print("Reconstructed secret:", reconstructSecret(pool)) 



Original Secret: 1234

Shares: [74143, 146447308615340] [56826, 86027509735246] [99512, 263809387121058] [22219, 13152434443892] [76964, 157803305336682]

Combining shares: [22219, 13152434443892] [76964, 157803305336682] [56826, 86027509735246]
Reconstructed secret: 1234


## Ramp Secret Sharing

In [5]:
import rs_sss
import numpy as np

In [6]:
deg = 8 # m of GF(2^m)
s = rs_sss.RS_SSS(deg)

In [7]:
threshold = 8 # k
ramp = 3  # L
num = 11  # n
s.initialize(threshold, ramp, num)

In [8]:
# generate a random secret of 1024 bytes (deg = 8)
orig_size = 1024
orig_secret = np.random.randint(0, (1 << deg) - 1, orig_size) 

# set the secret into the instance
s.set_secret(orig_secret) 

In [9]:
# generate shares from the given secret
s.generate_shares()

# shares are generated in SSS._shares[]
for i in range(num):
    print("Share {0}: {1}".format(i, s.get_shares()[i]))

Share 0: [ 17 223  39  22 119 222   2  77 253  21 221   3   1 174  78   4 148  66
  11  90   1  97  93 102 154 169 118  89 104 250 245 177 155 249  74 238
 126 231 209  37  47  88 252  63 130  35 113 188 224 248  78 135 252 133
  73 164 168 187 230 170  97  57   2 151  23 170 173  14  10  54  58 177
 160  42 252 164 218  48 243  67   9  75 190 152 150  62   0 200 255 136
 173  72 173 253 131 133 187 104 198  19 239 120  17 137 190 198  26 192
  43 155 191 107 135 227 206 177 254 187   0 226 199 166 167 181 253  97
 165 144 245  61 129 218   1  10 183 215  76 170  19 143  87 171 217 197
 182   8 179  52 175  76  58 117 175 133  92 127 165 178 159 185 249  51
  73  49  22 217   5 251 232 157 176 141 241 120 176   2 205 186 107 217
 160  68  59   2  98  83 149 247  28  76  35  64  83 152 167  90  80 176
   1  44  73  90  13   9 209 177  34 236 142 226 119 216 101  38 239 111
 155 196  27  21 164 206 146  32 204  82 120  74 187 206  12 227 232  59
 172 204  33 202  80 243 209 131  31 211 1

In [10]:
shares = []  # list of shares for secret reconstruction
index_list = [0, 1, 2, 8, 4, 7, 9, 10]  # list of share indices for secret reconstruction

# copy from the instance variable
for i in index_list:
    shares.append(s.get_shares()[i])
    
# initialize again and remove all data from the instance
s.__init__(deg)
s.initialize(threshold, ramp, num)

# set copied shares and their indices to the instance
s.set_external_shares(shares, index_list)

In [11]:
# reconstruct the secret
s.reconstruct_secret(orig_size)
reco_secret = s.get_secret()

# check if the original secret coincides with the reconstructed one
print("Original secret == Reconstructed secret ?: {0}".format(np.allclose(orig_secret, reco_secret)))

Original secret == Reconstructed secret ?: True


## Shamir Secret Sharing 

In [4]:
from Crypto.Protocol.SecretSharing import Shamir
from Crypto.Random import get_random_bytes
from binascii import hexlify
# Generating Key
key = get_random_bytes(16)
print("This is the Original Key: ", key)

# Sharing the key by spliting into 5 keys, we need 2 keys to get the original key. 
shares = Shamir.split(2, 5, key)
#for idx, share in shares:
    #print ("Index #%d: %s" % (idx, hexlify(share)))
    
shares1 = []
for i in shares:
    shares1.append(i)
    
# Combining 2 keys to get the original key
Shamir.combine([shares1[0], shares1[1]])
combined = Shamir.combine([shares1[0], shares1[1]])
print("This is the combined key: ", combined)

# Checking both keys are a match
print("True: Keys are a match, False: Keys are different")
print("If both keys are a match:")
print(combined==key)

This is the Original Key:  b"\x9b7\xff'\x90-\x8b\x88\x0b\xfa$\xcb\xdes\x17r"
This is the combined key:  b"\x9b7\xff'\x90-\x8b\x88\x0b\xfa$\xcb\xdes\x17r"
True: Keys are a match, False: Keys are different
If both keys are a match:
True


## Ramp Secret Sharing 

In [2]:
# Get the key
key = b'~s]m\xa8\xd6\xca8\x13\xce\x96\xca\x8e\xa7\x105\xd5\x80\xb4\x1fj\xd5\x0e5<AH\x8a\nU)\x18'

# Convert key to String
#t = key.decode('utf-8','ignore')


#### Trying to implment Ramp Secret Sharing Technique

In [13]:
# generate a random secret of 1024 bytes (deg = 8)
orig_size = 1024
orig_secret = np.random.randint(0, (1 << deg) - 1, orig_size)
print(orig_secret)
# set the secret into the instance
s.set_secret(orig_secret) 

[105   5  22 ...  45 152 150]


### Generating Shares from Secret Sharing 

In [14]:
# generate shares from the given secret
s.generate_shares()

# shares are generated in SSS._shares[]
for i in range(num):
    print("Share {0}: {1}".format(i, s.get_shares()[i]))

Share 0: [199 123 215 169 157 153 123 132  27 218 230 195  52 136 227  30 250  92
 220 238  74  87 226 255  64 186 243 101  18  88  98 214  38 217  42 184
  92  69  60 179 234 119  36  93 159 146 252 174 109 107 242 222 211  51
   4 117 127  36  71 107  94 156 213  13 205 173 194  20  65 235 151  18
 245  13 193 224 214 246 171  51 179 139 119 143 109  91  70 236 204  81
 136 236 116 128  84 108  63 240 225  89  70 235 150  73 168  35  80  33
 150 112 145 168 124 209 175 118 147 204 190 208 145 160 253  99  23  57
  75 174  26 175 140 206 210 223 162  93 212 197 149 148  18 193  25 116
 108  26  10   6 167 115  72 135 210 188 133  67 204 212   9 107 235 140
 232 203 183  10  15 182  91  29 191 184 215 110  44 140 242  47  80 176
  12  26 249  13 157  19 175  58 156  59 205 252  58  61  23 116  32 206
 194 120 235  35 121 199 180 124 189 121 200 199 190 134  85 198 197 115
  60  57 143 207 159  35  95 193  86 183  39  50 142  11 222 115 136 186
 246 145 102 154 197 211 216  77  66 155 1

### Another Implementation for Secret Sharing Scheme

##### Convert Bytes to Integer Array. 

In [15]:
key = '~s]m\xa8\xd6\xca8\x13\xce\x96\xca\x8e\xa7\x105\xd5\x80\xb4\x1fj\xd5\x0e5<AH\x8a\nU)\x18'